In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [2]:
transform = transforms.Compose([transforms.ToTensor()])  # Tensor로 바꿔야 입력이 가능해짐
train_data = torchvision.datasets.ImageFolder(root='custom_data/origin_data', transform=transform)

In [ ]:
data_loader = DataLoader(dataset=train_data, batch_size=8, shuffle=True, num_workers=2)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120, 2)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out

In [ ]:
# test code
net = CNN().to(device)
test_input = (torch.Tensor(3, 3, 64, 128)).to(device)
test_out = net(test_input)

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.00001)
loss_func = nn.CrossEntropyLoss().to(device)

In [ ]:
total_batch = len(data_loader)

epochs = 3
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()

        avg_cost += loss / total_batch

    print(f'[Epoch:{epoch + 1}] cost = {avg_cost}')
print('Learning Finished!')

In [ ]:
torch.save(net.state_dict(), "./model/model.pth")

In [ ]:
new_net = CNN().to(device)

In [ ]:
new_net.load_state_dict(torch.load('./model/model.pth'))

In [ ]:
print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

In [ ]:
trans=torchvision.transforms.Compose([transforms.Resize((64,128)), transforms.ToTensor()])
test_data = torchvision.datasets.ImageFolder(root='./custom_data/test_data', transform=trans)

In [ ]:
test_dataset = DataLoader(dataset=test_data, batch_size=len(test_data))

In [ ]:
with torch.no_grad():
    for num, data in enumerate(test_dataset):
        imgs, labels = data
        imgs = imgs.to(device)
        label = label.to(device)

        prediction = net(imgs)

        correct_prediction = torch.argmax(prediction, 1) == label

        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())